In [ ]:
import numpy as np
from numpy import pi
from scipy.interpolate import CubicSpline
from scipy.sparse.linalg import LinearOperator, gmres
from tqdm import tqdm

from derivative.finiteDifferenceMethods import centralFDM
from integrate.lowStorageRungeKuttaMethods import LSRK3
from utils.grid import wallNormalGrid
from utils.wallLaw import wallLow

In [ ]:
class channel2(object):
    Re_tau = NotImplemented
    nx = NotImplemented
    ny = NotImplemented
    lx = NotImplemented
    nu = NotImplemented
    delta = NotImplemented
    eta = NotImplemented
    dt = NotImplemented
    def __init__(self):
        self.x = np.linspace(0, self.lx, self.nx, endpoint=False)
        self.y = wallNormalGrid(self.ny, self.eta, self.delta)
        self.X, self.Y = np.meshgrid(self.x, self.y)
        self.yPlus = (self.delta - np.abs(self.y)) / self.delta * self.Re_tau
        self.u_tau = 1. / wallLow(self.Re_tau)
        self.nu = self.u_tau * self.delta / self.Re_tau
        self.dx = self.lx / self.nx
        self.integrate = LSRK3()
        self.derivative = centralFDM(order=4, highestDerivative=2)
        # self.force = wallLow(self.Re_tau) ** -2
        self.force = 0.
        self.p_shape = (self.ny, self.nx)
        self.A_shape = (self.nx*self.ny, self.nx*self.ny)
        self.exitCode = 0
        self.A = LinearOperator(self.A_shape, matvec=self.laplacian_p)

    def rhs(self, t, u):
        u_interp = CubicSpline(self.y, u, axis=-2, bc_type='not-a-knot')
        return ( - u[0] * self.derivative(u, h=self.dx) 
                 - u[1] * u_interp(self.y, nu=1)
                 + self.nu * (self.derivative(u, h=self.dx, derivative=2) 
                              + u_interp(self.y, nu=2))
                 + self.force )

    def laplacian_p(self, p_flatten):
        p_ = np.reshape(p_flatten, self.p_shape)
        p_interp = CubicSpline(self.y, p_, axis=-2, bc_type='clamped')
        p_ = self.derivative(p_, h=self.dx, derivative=2) + p_interp(self.y, nu=2)
        return p_.flatten()

    def div_u(self, u):
        u2_interp = CubicSpline(self.y, u[1], axis=-2, bc_type='not-a-knot')
        return (self.derivative(u[0], h=self.dx)
                + u2_interp(self.y, nu=1))

    def grad_p(self):
        p_interp = CubicSpline(self.y, self.p_, axis=-2, bc_type='clamped')
        return np.stack((self.derivative(self.p, h=self.dx) , p_interp(self.y, nu=1)))

    def step(self):
        u = self.integrate.step(fun=self.rhs, u=self.u, t0=self.t, dt=self.dt)
        self.u[:,(0,-1),:] = 0.
        self.p_, self.exitCode = gmres(A=self.A, b=self.div_u(u).flatten()/self.dt, x0=self.p_.flatten())
        self.p_ = np.reshape(self.p_, self.p_shape)
        self.u -= self.dt * self.grad_p()
        self.u[:,(0,-1),:] = 0.
        self.p = self.p_ - self.p

    def solve(self, u0, t0, ts_span, p0=None):
        self.u = u0
        self.t = t0
        if p0 == None:
            self.p = np.zeros(self.p_shape)
        else:
            self.p = p0
        self.p_ = self.p
        for ts in tqdm(range(ts_span[0], ts_span[1]+1)):
            self.step()
            self.t = t0 + self.dt*ts

In [ ]:
class channel2_100(channel2):
    Re_tau = 100
    nx = 168
    ny = 127
    lx = 2. * pi
    nu = 1.
    delta = 1.
    eta = 0.94
    dt = 0.004


# class channel2_180(channel2):
#     Re_tau = 180
#     nx = 256
#     ny = 191
#     lx = 2. * pi
#     nu = 1.
#     delta = 1.
#     eta = 0.95
#     dt = 0.002


# class channel2_400(channel2):
#     Re_tau = 400
#     nx = 384
#     ny = 255
#     lx = 2. * pi
#     nu = 1.
#     delta = 1.
#     eta = 0.97
#     dt = 0.001

In [ ]:
class initializeVelocityField(object):
    def __init__(self, nx, ny, lx, delta, eta):
        self.nx = nx
        self.ny = ny
        self.lx = lx
        self.delta = delta
        self.eta = eta
        self.x = np.linspace(0, self.lx, self.nx, endpoint=False)
        self.y = wallNormalGrid(self.ny, self.eta, self.delta)
        self.X, self.Y = np.meshgrid(self.x, self.y)
        self.dx = self.lx / self.nx
        self.derivative = centralFDM(order=4, highestDerivative=2)
        self.u_shape = (self.ny, self.nx)
        self.A_shape = (self.nx*self.ny, self.nx*self.ny)
        self.exitCode = 0
        self.A = LinearOperator(self.A_shape, matvec=self.lhs)

    def lhs(self, v_flatten):
        v = np.reshape(v_flatten, self.u_shape)
        v[(0,-1),:] = 0.
        v_interp = CubicSpline(self.y, v, axis=-2, bc_type='not-a-knot')
        return v_interp(self.y, nu=1).flatten()

    def rhs(self, u):
        return - self.derivative(u, h=self.dx, derivative=1).flatten()

    def initialize_v(self, u):
        v, self.exitCode = gmres(A=self.A, b=self.rhs(u))
        return np.reshape(v, self.u_shape)

In [ ]:
tcf = channel2_100()
u0 = np.expand_dims(wallLow(tcf.yPlus), axis=1) * np.ones((tcf.ny, tcf.nx)) * tcf.u_tau
u0 *= np.random.normal(loc=1., scale=0.01, size=u0.shape)
initilizer = initializeVelocityField(nx=tcf.nx, ny=tcf.ny, lx=tcf.lx, delta=tcf.delta, eta=tcf.eta)
v0 = initilizer.initialize_v(u=u0)
U0 = np.stack((u0, v0))
t0 = 0.
ts_span = (0, 100)
# tcf.solve(U0, t0, ts_span)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.contourf(tcf.X, tcf.Y, u0, levels=100, cmap="hot")
# plt.contourf(tcf.X, tcf.Y, tcf.p, levels=100, cmap="hot")
# plt.contourf(tcf.X, tcf.Y, tcf.u[0], levels=100, cmap="hot")
plt.colorbar()
plt.show()